### Analyze FVA for models under certain regression conditions

In [ ]:
import os.path as path
import glob, os
from datetime import datetime
from importlib.metadata import version
import cobra
import thermo_flux
from thermo_flux.io import load_excel as ex
from thermo_flux.io import helper_load as hl
from thermo_flux.core.model import ThermoModel
from thermo_flux.solver.gurobi import variability_analysis, variability_results
from equilibrator_api import  Q_
import pandas as pd
import numpy as np
from scripts.logger import write_to_log
import gurobipy as gp
from gurobipy import GRB
from scripts.gen_model import gen_model
from scripts.gen_model import apply_physio_data
from scripts.gen_model import constrain_bounds_fva
from scripts.reaction_utils import list_and_remove_blocked_reactions
from scripts.reaction_utils import list_blocked_reactions
from scripts.reaction_utils import count_blocked_pathways

In [ ]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

OUTPUT_DIR = "output"
OUTPUT_NAME = f"thermo_fva_test"
OUTPUT_LOG = f"{OUTPUT_DIR}{path.sep}{OUTPUT_NAME}_log.txt"

CONDITIONS_TO_REGRESS = ["WT-Glc_I", "WT-Gal_I", "WT-Fruc_I", "WT-Mann_I", "dptsG-Glc_I", 
                         "WT-Ace_I", "WT-Succ_I", "WT-Fum_I", "WT-Glyc_I", "WT-Pyr_I",
                         "WT-GlyCAA_II"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
write_to_log(OUTPUT_LOG, f"Started analysis at: {time}", "w")

# Write package versions:
modules = ["pandas", "numpy", "equilibrator_api", "cobra"]
write_to_log(OUTPUT_LOG, f"Package versions used:")
versions_packages = [f"  {m}: {version(m)}\n" for m in modules]
write_to_log(OUTPUT_LOG, "".join(versions_packages))

In [ ]:
tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)

In [ ]:
blocked = list_blocked_reactions(tmodel, "BASE", OUTPUT_LOG, 1)
print(len(blocked))

In [ ]:
tmodel = apply_physio_data(tmodel, CONDITIONS_TO_REGRESS[0], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, ALLOW_OTHER_EXCRETION, OUTPUT_LOG)

In [ ]:
tmodel.remove_reactions(blocked)
for rxn in tmodel.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel.update_thermo_info(fit_unknown_dfG0=True)

In [ ]:
tmodel.m = None  
tmodel.objective = tmodel.reactions.biomass_EX  
tmodel.add_TFBA_variables() 

In [ ]:
rxn_ids = [r.id for r in tmodel.reactions]
indices = []
for rxn_id in rxn_ids:
    rxn = tmodel.reactions.get_by_id(rxn_id)
    idx = tmodel.reactions.index(rxn)  # ← manual index
    indices.append(idx)
vars_to_analyze = [tmodel.mvars["v"][0, idx] for idx in indices]
print(vars_to_analyze)

In [ ]:
vars = vars_to_analyze
gm = variability_analysis(tmodel, vars)
tmodel.m = gm

In [ ]:
print(tmodel.m)

In [ ]:
tmodel.m.Params.TimeLimit = 7200
tmodel.m.Params.Threads = 16    
tmodel.m.optimize()

In [ ]:
gm_results = variability_results(tmodel)

In [ ]:
tmodel.remove_reactions(blocked)

In [ ]:
tmodel = constrain_bounds_fva(tmodel, OUTPUT_LOG)